In [1]:
import numpy as np
import matplotlib.pyplot as plt
import CTS as cts
from scipy.constants import *
from scipy import integrate
from scipy import fft
from matplotlib.lines import Line2D
from plasmapy.formulary.frequencies import plasma_frequency
from plasmapy.particles import Particle

plt.rcParams.update({'font.size': 20})
%matplotlib qt

In [ ]:
# Parameters
f0 = 330e9  # 330 GHz center frequency
n_cycles = 16
n_e = 5e12  # cm^-3
L = 1  # cm
bandwidth_factor = 0.3  # Show frequencies within 30% of f0
mass = Particle("e-").mass.value
omega_p = plasma_frequency.lite(n_e*1e6, mass, Z=1, to_hz=False)

In [21]:
# Generate the wave packet
# t, signal, freqs, signal_fft, envelope = cts.generate_pulse_wave_packet(f0, n_cycles, num_points=4096, return_envelope=True)
t, signal, freqs, signal_fft, envelope = cts.generate_n_cycle_wave_packet(f0, n_cycles, return_envelope=True) 

omega = 4 * np.pi * freqs

# Create a wrapper function that can be passed as the dispersion relation
dispersion_func = lambda omega: cts.plasma_dispersion_relation(omega, n_e*1e6)

# Now pass the function, not its result
fft_propagated = cts.propagate_through_dispersive_medium(freqs, signal_fft, L/100, dispersion_func)

# Convert propagated signal back to time domain
signal_propagated = fft.ifft(fft_propagated)

In [22]:
# Create figure with two subplots
fig, axs = plt.subplots(2, 1, figsize=(10, 8))

# Define frequency range of interest around f0
f_min = f0 * (1 - bandwidth_factor)
f_max = f0 * (1 + bandwidth_factor)

# Create masks for frequencies within the range of interest
# First for positive frequencies
pos_mask = (freqs >= f_min) & (freqs <= f_max)
# Also for negative frequencies (corresponding to -f0)
neg_mask = (freqs >= -f_max) & (freqs <= -f_min)

# Plot frequency domain (FFT magnitude) - only around f0 using scatter
axs[0].scatter(freqs[pos_mask]/1e9, np.abs(signal_fft[pos_mask]), 
                c='b', s=20, alpha=0.6, label='Original')
axs[0].scatter(freqs[pos_mask]/1e9, np.abs(fft_propagated[pos_mask]), 
                c='r', s=20, alpha=0.6, label='After propagation')
axs[0].set_xlabel('Frequency (GHz)')
axs[0].set_ylabel('FFT Magnitude')
axs[0].set_title(f'Wave Packet Spectrum around {f0/1e9:.1f} GHz')
axs[0].legend()
axs[0].grid(True)

# Add vertical line at center frequency
axs[0].axvline(x=f0/1e9, color='k', linestyle='--', label=f'f0 = {f0/1e9:.2f} GHz')

# Plot time domain signals
axs[1].plot(t*1e9, signal.real, 'b-', label='Original')
axs[1].plot(t*1e9, signal_propagated.real, 'r-', label='After propagation')
axs[1].set_xlabel('Time (ns)')
axs[1].set_ylabel('Amplitude')
axs[1].set_title(f'Wave Packet Time Domain (n_e={n_e:.1e} cm^-3, L={L} cm)')
axs[1].legend()
axs[1].grid(True)

plt.tight_layout()
plt.show()

In [2]:
freq_in = [330e9,600e9] # Hz
cl = ['r', 'g', 'b']

ne = [1e12, 1e13, 5e13]
Te = [5,5, 300]
Ti = [1,1, 100]

In [10]:
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_xlabel(r"$\omega_s - \omega_i (rad/s)$")
ax.set_ylabel(r"$S(k, \omega)$")

for i in range(len(ne)):

    lambda_in = c / freq_in[0]
    omega_in = 2*np.pi*freq_in[0]

    alpha, omega_arr, omega_in, Skw= cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=10000, ions="He+")
    
    ax.plot(omega_arr-omega_in, Skw, label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}", color=cl[i])

    a =cts.compute_alpha(lambda_in*100, 180, ne[i], Te[i])
    print('S integral = ',integrate.simpson(Skw, omega_arr))
    print('alpha = ', a)

    lambda_in = c / freq_in[1]
    omega_in = 2*np.pi*freq_in[1]

    alpha, omega_arr, omega_in, Skw = cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=1000, ions="He+")
    ax.plot(omega_arr-omega_in, Skw, '--', color=cl[i])


# Create first legend for the data lines
first_legend = ax.legend(loc='upper right', fontsize=16, bbox_to_anchor=(1, 1))
# Add the first legend manually to the axes
ax.add_artist(first_legend)
# Add a second legend for line styles
custom_lines = [Line2D([0], [0], color='black', lw=2),
                Line2D([0], [0], color='black', lw=2, linestyle='--')]
second_legend = ax.legend(custom_lines, [f'{freq_in[0]/1e9:.0f} GHz', f'{freq_in[1]/1e9:.0f} GHz'], 
                          loc='upper left', fontsize=20, bbox_to_anchor=(0, 1))
plt.tight_layout()

# fig, ax = plt.subplots(figsize=(16, 6))
# ax.set_xlabel(r"frequency (GHz)")
# ax.set_ylabel(r"$S(k, \omega)$")

# for i in range(len(ne)):
#     alpha, omega_arr, Skw = cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=5000, ions="He+")
#     freq_arr = omega_arr/(2*np.pi) *1e-9
#     ax.plot(freq_arr, Skw, '--',label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}")
    
# ax.legend(loc='upper left')
# ax.set_title("Helium")

S integral =  -1.2034781662586662
alpha =  4.387786790594731
S integral =  -1.2754846909468842
alpha =  13.87540014547963
S integral =  -1.5441085723419812
alpha =  4.005483004553218


In [ ]:
lambda_in = 2*np.pi*c / omega_arr

plt.figure(figsize=(8, 6))
for i in range(len(ne)):
    a_ls =cts.compute_alpha(lambda_in*100, 180, ne[i], Te[i])
    plt.plot(omega_arr*1e-9, a_ls, label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}")

plt.xlabel(r"frequency (GHz)")
plt.ylabel(r"$\alpha=1/k\lambda_D$")
plt.yticks(fontweight='bold')
plt.legend(loc='upper right', fontsize=16)
plt.tight_layout()

In [18]:
freq_ls = np.arange(300e9, 600e9, 10e9)
plt.figure(figsize=(8, 5))
plt.xlabel("frequency (GHz)")
plt.ylabel("$P_s/P_iL (1/cm) $")

for i in range(len(ne)):
    pr_ls = []
    for freq in freq_ls:

        lambda_in = c / freq
        alpha, omega_arr, omega_in, Skw = cts.generate_spectral_density(lambda_in*1e9, Te[i], Ti[i], ne[i], delta_lam=5000, ions="H+")
        pr = cts.power_ratio(ne[i], omega_arr, Skw, scattering_angle=180, L=1)
        pr_ls.append(pr)

    plt.plot(freq_ls*1e-9, pr_ls, label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}")
plt.legend()
plt.tight_layout()

In [4]:
plt.figure(figsize=(8, 6))
for i in range(len(ne)):
    rot_angle = cts.faraday_rotation_angle(freq_ls, ne[i], 1, 1)
    plt.plot(freq_ls*1e-9, rot_angle, label=f"ne={ne[i]:.1e}, Te={Te[i]}, Ti={Ti[i]}")
plt.legend(loc='upper right', fontsize=14)
plt.title("Faraday rotation angle per cm per Gauss")
plt.xlabel("frequency (GHz)")
plt.ylabel("deg/(cm*Gauss)")
plt.tight_layout()